## Interfaces con otros lenguajes: C

Existen varias formas de utilizar bibliotecas o códigos hechos en C desde Python. Nosotros veremos el uso de `Ctypes`, sin embargo existen otras alternativas como  [Cython](https://cython.org/), [CFFI](https://cffi.readthedocs.io/en/latest/), [pybind11](https://pybind11.readthedocs.io/en/stable/) y [Boost.Python](https://www.boost.org/doc/libs/1_70_0/libs/python/doc/html/index.html). 


Seguimos con el ejemplo de la rotación de vectores. Por completitud, agregamos la funciones en Python para comparar


In [ ]:
import numpy as np

In [ ]:
pwd

Si ya tenemos un módulo donde están programadas las funciones necesarias 

In [ ]:
import numpy as np

def matrix_rotation(angles):
  cx, cy, cz = np.cos(angles)
  sx, sy, sz = np.sin(angles)
  R = np.zeros((3, 3))
  R[0, 0] = cx * cz - sx * cy * sz
  R[0, 1] = cx * sz + sx * cy * cz
  R[0, 2] = sx * sy

  R[1, 0] = -sx * cz - cx * cy * sz
  R[1, 1] = -sx * sz + cx * cy * cz
  R[1, 2] = cx * sy

  R[2, 0] = sy * sz
  R[2, 1] = -sy * cz
  R[2, 2] = cy
  return R


def rotate(angles, v):
  return np.dot(matrix_rotation(angles), v)


In [ ]:
N = 100
# Ángulos de Euler
angle = np.random.random(3)
# Definimos N vectores tridimensionales
v = np.random.random((3, N))

In [ ]:
# y= rotp.rotate(angle, v)
y = rotate(angle,v)

In [ ]:
print(angle)
print(y[:,0:5].T)

Veamos cómo trabajar si tenemos el código para realizar las rotaciones en C.

### Primer ejemplo: Nuestro código

El código en C que tenemos es:

```c
 typedef struct {
      float m[3][3];
    } m3x3;

    typedef struct {
      float a[3];
    } v3;
    
...

float * rotate(float angles[3], float *v, int N){

        m3x3 R = matrix_rotation(angles);
        
        float* y = (float*)malloc(3*N*sizeof(float));
        v3 p;

        printf("%p\n",y);
        for(int i=0; i<N; i++){
            // p = &y[i*3];
            p = matmul3(R,&v[i*3]);
            y[i*3+0] = p.a[0];
            y[i*3+1] = p.a[1];
            y[i*3+2] = p.a[2];
            // printf("%6.3f %6.3f %6.3f \n",y[i*3+0],y[i*3+1],y[i*3+2]);
        }
        return y;


  }

```

In [ ]:
cd ../scripts/interfacing_C

### CTypes

No vamos a usar directamente `Ctypes`, sino a través de `NumPy`, que provee algunas funciones convenientes para acceder al código C.

El primer paso es compilar nuestro código y generar una biblioteca:
```bash
$ gcc -fpic -Wall -shared rotacion.c -o librotacion.so
```

Si uno trabaja en Windows con [MS C++](https://visualstudio.microsoft.com/es/vs/features/cplusplus/), generará una dll

```cmd
cl.exe -c rotacion.c
link.exe /DLL /OUT:rotacion.dll
```

Si en cambio [instaló gcc](https://www.msys2.org/), puede usar el comando previo.


In [ ]:
!gcc -fpic -Wall -shared rotacion.c -o librotacion.so

In [ ]:
!ls

En segundo lugar, importamos el módulo `ctypeslib`

In [ ]:
import numpy.ctypeslib as ctl

Este módulo nos provee de la función `load_library` para importar la biblioteca

In [ ]:
help(ctl.load_library)

In [ ]:
rotc = ctl.load_library('librotacion.so','.')

Una vez cargada la biblioteca, tenemos que definir adecuadamente cómo pasar los argumentos a la función `rotate` de C:

```C
    float * rotate(float angles[3], float *v, int N)
```

Para eso se utiliza la función `argtypes` que recibe una lista de tipos. Notemos que los dos primeros argumentos son arreglos de C (o sea, punteros), mientras que el último es un entero.

In [ ]:
npflags = ['C_CONTIGUOUS']   # Require a C contiguous array in memory

float_1d_type = ctl.ndpointer(dtype=np.float32, ndim=1, flags=npflags) # Puntero a float, 1D
float_2d_type = ctl.ndpointer(dtype=np.float32, ndim=2, flags=npflags) # Puntero a float, 2D

In [ ]:
type(float_1d_type)

Con estos tipos de datos, defino los tipos de argumentos, que son tres en total. El último es un dato de tipo entero, para lo cual se usa directamente `c_intp`. Para definir el tipo de argumentos de entrada a la función `rotc.rotate` usamos el método `argtypes`: 

In [ ]:
rotc.rotate.argtypes =  [float_1d_type, float_2d_type, ctl.c_intp]

Hagamos un ejemplo sencillo con N=2

In [ ]:
N = 2
# Ángulos de Euler
angle = np.random.random(3).astype(np.float32)
# Definimos N vectores tridimensionales
v = np.random.random((3, N)).astype(np.float32)

Las funciones que dispongo en C reciben tipos `float`, es decir que me tengo que asegurar esto a través del método `astype`.

Ahora tenemos que definir el tipo de dato de salida, que retorna C a través de un puntero a float, `float*`. Para esto usamos el método `restype`. Como a priori no sé qué tipo de rango tiene mi arreglo de salida, tengo que definirlo explícitamente. 

In [ ]:
rotc.rotate.restype = ctl.ndpointer(dtype=np.float32, shape=(N,3)) 

Hay que tener precaución con el manejo de arreglos, que es muy distinto en C y Python. En Python son objetos, de los cuales yo puedo tener distintas vistas, slices, etc. Hay que recordar que en principio estas son formas de acceder al mismo objeto, pero no se pueden traducir directamente a C, que necesita un arreglo contiguo de datos.

In [ ]:
v = np.array([[1,0], [0,1], [0,0]]).astype(np.float32) 
vt = v.T.copy()

print(v)
print(vt)
print(np.shape(v))
print(np.shape(v.T))

Veamos, v es un arreglo de 3 filas y 2 columnas, que contiene *dos* vectores de tres dimensiones que se desean rotar, organizados como columnas. Esto *no* es lo que necesita mi arreglo en C, que es tiene los vectores organizados contiguamente en un solo arreglo unidimensional. Entonces, tengo que transformarlo. Para eso usamos el `.T`. Ojo que además, hay que crear un objeto nuevo con `copy()`, sino es una vista del mismo objeto `v`.

In [ ]:
angle90 = np.array([0,0,np.pi/2],dtype = np.float32)
print(angle90)

In [ ]:
yf = rotc.rotate(angle90,
                      vt,
                      N) 
y = rotate(angle90,v)

In [ ]:
np.set_printoptions(suppress=True) # suppress controla cómo print va a escribir los números de punto flotante.

print(y)
print(yf.T)
np.allclose(y,yf.T)